## **Carga de librerias**

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

## **Carga de los datos**
Se definió un subconjunto con **1464** registros a partir de los **48700** registros iniciales. Esta división se llevó a cabo dado que se encontró una gran disparidad de datos en la columna que sería la etiqueta de salida (Se encontraron aproximadamente el triple de respuestas **NO** en contraposición a respuestas **SI**). El subconjunto obtenido se mezcló de manera aleatoria para evitar problemas de convergencia al iniciar el entrenamiento. Adicionalmente, del conjunto inicial se extrajo 200 registros para pruebas con el sistema desplegado.


In [ ]:
# Load data
##############################################################################
data = pd.read_csv('data.csv', encoding='utf-8')

subsetYes = data[data['f2_s2_200'] == 'si'].sample(frac=1) # Yes registries
subsetYes.reset_index(inplace=True, drop=True)

subsetTestYes = subsetYes[1364:] # Registries to test the system
subsetYes = subsetYes[:1364]

subsetNo = data[data['f2_s2_200'] == 'no'].sample(frac=1) # No registries
subsetNo.reset_index(inplace=True, drop=True)

subsetTestNo = subsetNo[1364:1464] # Registries to test the system
subsetNo = subsetNo[:1364]

data = pd.concat([subsetYes, subsetNo], axis=0).sample(frac=1)
data.reset_index(inplace=True, drop=True)

dataTest = pd.concat([subsetTestYes, subsetTestNo], axis=0)
dataTest.reset_index(inplace=True, drop=True)
dataTest.to_csv('registriesToTest.csv', index=None)

## **Preprocesamiento**
Para empezar, se removieron las columnas que no guardaban relación con la problemática tratada. Se utilizó codificación **OneHot** para los datos categóricos y un **escalamiento estándar** para las edades.

**Remplazo de valores**

Se remplazo el valor de las celdas vacías o cuyo valor era NaN, pues al revisar el diccionario de datos se concluyó que las celdas en blanco en su mayoría representaban falta de conocimiento  o desinterés por contestar la pregunta.

**División del conjunto (entrenamiento y prueba)**

Se definió la división de los datos con un 30% para el conjunto de pruebas


In [ ]:
# Preprocesing
##############################################################################
Xenc = OneHotEncoder()
scaler = StandardScaler()
lencoder = LabelEncoder()
omisiones = [
    'prov', 'f2_s1_101', 'f2_s6_601_1', 'f2_s6_603_12',
    'f2_s8_804', 'f2_s8_806', 'f2_s8_807', 'f2_s8_808', 
    'f2_s8_809','f2_s8_814', 'f2_s8_821', 'gedad_anios', 
    'estrato', 'f2_s2_200', 'f2_s2_207'
]

for x in data.columns:
    data[x] = data[x].fillna('no sabe / no responde')
    
X = data.drop(omisiones, axis=1)

y = lencoder.fit_transform(data['f2_s2_200'])

Xenc = Xenc.fit(X)

XE = Xenc.transform(X).toarray()
XE = np.concatenate(
    (XE, scaler.fit_transform(pd.DataFrame(data['f2_s1_101']))), 
    axis=1
)

xTrain, xTest, yTrain, yTest = train_test_split(
    XE, y, test_size=0.30, random_state= 0
)


## **Entrenamiento**

In [ ]:
# Training
#############################################################################
classifier = LogisticRegression(solver='liblinear')
classifier.fit(xTrain, yTrain)


## **Predicciones**

In [ ]:
# Predictions
##############################################################################
predictions = classifier.predict(xTest)
print(classifier.score(xTest, yTest))


## **Almacenamiento del modelo y los codificadores**

In [ ]:
# Saving model and encoders
##############################################################################
pickle.dump(scaler, open('scaler.sav', 'wb'))
pickle.dump(Xenc, open('featuresEncoder.sav', 'wb'))
pickle.dump(classifier, open('classifier.pkl', 'wb'))


## **Resultados y metricas**

Se utilizó una matriz de confusión para expreser la exactitud del modelo al utilizar los datos de prueba.

In [ ]:
# Confusion matrix
##############################################################################
cm = confusion_matrix(yTest, predictions)
display = ConfusionMatrixDisplay(confusion_matrix = cm)
display.plot()
plt.savefig("cm.svg", format='svg')

# K-fold cross validation
##############################################################################
accuracies = cross_val_score(
    estimator = classifier, X = xTrain, y=yTrain, cv=10)

print(accuracies.mean())
